# Weather Dataset Preprocessing

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns


from utils import *

datasets_folder = './datasets'

In [3]:
weather_folder = os.path.join(datasets_folder,'weather')

weather_df = merge_csv_to_dataframe(weather_folder).rename(columns={'PragaTime':'Date'})
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df

Merged 6 CSV files


,Date,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,W_VEC_INT,LEAFW,ET0
0,2019-01-01 00:00:00,1.0,0.0,92.3,0.0,0.5,208.7,0.3,0.0,0.0
1,2019-01-01 01:00:00,0.3,0.0,93.6,0.0,0.5,280.0,0.2,0.0,0.0
2,2019-01-01 02:00:00,0.7,0.0,91.7,0.0,1.1,158.1,1.0,0.0,0.0
3,2019-01-01 03:00:00,0.4,0.0,91.5,0.0,0.7,189.4,0.4,0.0,0.0
4,2019-01-01 04:00:00,1.0,0.0,86.3,0.0,0.7,172.9,0.6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
52580,2024-12-30 20:00:00,6.2,0.0,72.1,0.0,2.0,268.9,1.6,0.0,0.0
52581,2024-12-30 21:00:00,5.6,0.0,76.1,0.0,2.5,258.0,2.4,0.0,0.0
52582,2024-12-30 22:00:00,5.7,0.0,75.3,0.0,2.8,254.1,2.7,0.0,0.0
52583,2024-12-30 23:00:00,5.1,0.0,76.1,0.0,2.8,256.7,2.7,0.0,0.0


The last 23 hours of the 2024 are missing...

There are no missing values nor missing hours inbetween:

In [4]:
weather_df.resample('1h', on='Date')
weather_df.isna().sum()

Date          0
TAVG          0
PREC          0
RHAVG         0
RAD           0
W_SCAL_INT    0
W_VEC_DIR     0
W_VEC_INT     0
LEAFW         0
ET0           0
dtype: int64

README: should we normalize sometihng?

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns_to_normalize = weather_df.columns.difference(['Date'])
weather_df[columns_to_normalize] = scaler.fit_transform(weather_df[columns_to_normalize])
weather_df

,Date,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,W_VEC_INT,LEAFW,ET0
0,2019-01-01 00:00:00,0.096698,0.0,0.911899,0.0,0.034247,0.579722,0.020690,0.0,0.0
1,2019-01-01 01:00:00,0.080189,0.0,0.926773,0.0,0.034247,0.777778,0.013793,0.0,0.0
2,2019-01-01 02:00:00,0.089623,0.0,0.905034,0.0,0.075342,0.439167,0.068966,0.0,0.0
3,2019-01-01 03:00:00,0.082547,0.0,0.902746,0.0,0.047945,0.526111,0.027586,0.0,0.0
4,2019-01-01 04:00:00,0.096698,0.0,0.843249,0.0,0.047945,0.480278,0.041379,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
52580,2024-12-30 20:00:00,0.219340,0.0,0.680778,0.0,0.136986,0.746944,0.110345,0.0,0.0
52581,2024-12-30 21:00:00,0.205189,0.0,0.726545,0.0,0.171233,0.716667,0.165517,0.0,0.0
52582,2024-12-30 22:00:00,0.207547,0.0,0.717391,0.0,0.191781,0.705833,0.186207,0.0,0.0
52583,2024-12-30 23:00:00,0.193396,0.0,0.726545,0.0,0.191781,0.713056,0.186207,0.0,0.0
